# Análise de Dados Eleitorais Manaus 1o turno

Meus contatos: https://linktr.ee/fmasanori

Agradecimentos à primeira turma do curso Desenvolvimento em Software Multiplataforma, da FATEC São José dos Campos, que desenvolveu este projeto, inicialmente para a [TV Vanguarda](http://vanguarda.tv/). Em especial à Monique Carniello.

ATENÇÃO: na primeira tela de instalação do Python, habilite a opção PATH, note que **não** é o padrão.

Instalar o Python 3.6 ou superior (sugerimos não usar Anaconda) <br>
https://python.org.br/instalacao-windows/ <br>
https://python.org.br/instalacao-linux/ <br>
https://python.org.br/instalacao-mac/ <br>

Após instalar o Python, abra o terminal (Linha de Comando) e instale as bibliotecas necessárias:<br>
<br>
**pip install pandas matplotlib jupyter numpy**

Arquivos necessários:<br> 
https://cdn.tse.jus.br/estatistica/sead/odsele/perfil_eleitor_secao/perfil_eleitor_secao_ATUAL_AM.zip
https://cdn.tse.jus.br/estatistica/sead/eleicoes/eleicoes2022/buweb/bweb_1t_AM_051020221321.zip<br>
<br>
**Descompactar em c:\users\seunome**<br>
<br>

# Primeira parte: Perfil do Eleitorado
<br>
<br>


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
estado = pd.read_csv("perfil_eleitor_secao_ATUAL_AM.csv", 
                 usecols = '''NR_ZONA NR_SECAO NM_MUNICIPIO DS_FAIXA_ETARIA DS_ESTADO_CIVIL DS_GRAU_ESCOLARIDADE 
                 QT_ELEITORES_INC_NM_SOCIAL QT_ELEITORES_PERFIL'''.split(),
                 sep=";", on_bad_lines = 'skip', encoding='Latin 1')
municipio = estado.query('NM_MUNICIPIO == "MANAUS"')
municipio = municipio.drop(columns = 'NM_MUNICIPIO')

In [ ]:
municipio.sample(20)

## Faixa Etária 


In [ ]:
FaixaEtaria = municipio.groupby(['DS_FAIXA_ETARIA'])['QT_ELEITORES_PERFIL'].sum()
FaixaEtaria

In [ ]:
FaixaEtaria.plot(kind='barh', figsize=(16,10))
plt.ylabel('FAIXA ETÁRIA')
plt.grid(linestyle = '--', linewidth = 0.5)


## Agrupamento da faixa etária

In [ ]:
Jovem = FaixaEtaria[1:8] 
Adulto = FaixaEtaria[8:14] 
Idoso = FaixaEtaria[14:22] 

In [ ]:
FaixasIdade = pd.DataFrame({'Manaus':[Jovem.sum(), Adulto.sum(), Idoso.sum() + int(FaixaEtaria[0])]},
                        #somando idosos acima de 100 anos que estão no ínício
                        index = ['Jovens até 29 anos', 'Adultos', 'Idosos a partir de 60'])
FaixasIdade

In [ ]:
FaixasIdade.plot(kind='barh', figsize=(16,8))
plt.ylabel('FAIXA ETÁRIA AGRUPADA')
plt.grid(linestyle = '--', linewidth = 0.5)


## Zonas Eleitorais com mais jovens e mais idosos

In [ ]:
jovens = {}
for zona in municipio.NR_ZONA.unique():
    result = municipio.query('NR_ZONA == @zona').groupby('DS_FAIXA_ETARIA')['QT_ELEITORES_PERFIL'].sum()
    jovens[zona] = result[1:8].sum()
for zona in sorted(jovens, key=lambda x:jovens[x], reverse = True):
    print (f'Zona eleitoral {zona}: {jovens[zona]}')

In [ ]:
idosos = {}
for zona in municipio.NR_ZONA.unique():
    result = municipio.query('NR_ZONA == @zona').groupby('DS_FAIXA_ETARIA')['QT_ELEITORES_PERFIL'].sum()
    idosos[zona] = result[14:22].sum() + int(result[0])
for zona in sorted(idosos, key=lambda x:idosos[x], reverse = True):
    print (f'Zona eleitoral {zona}: {idosos[zona]}')

## Estado Civil

In [ ]:
EstadoCivil = municipio.groupby('DS_ESTADO_CIVIL')['QT_ELEITORES_PERFIL'].sum().sort_values(ascending=False)
EstadoCivil

## Escolaridade


In [ ]:
Escolaridade = municipio.groupby(['DS_GRAU_ESCOLARIDADE'])['QT_ELEITORES_PERFIL'].sum().sort_values(ascending = False)
Escolaridade

## Salvar Dados

In [ ]:
FaixasIdade.to_csv('FaixasIdade.csv')
EstadoCivil.to_csv('EstadoCivil.csv')
Escolaridade.to_csv('Escolaridade.csv')

## Nome social


In [ ]:
NomeSocial = municipio.groupby(['QT_ELEITORES_INC_NM_SOCIAL'])['QT_ELEITORES_PERFIL'].count()
NomeSocial[1]

# Zonas Eleitorais com mais pessoas que utilizam nome social

In [ ]:
nome_social = {}
for zona in municipio.NR_ZONA.unique():
    result = municipio.query('NR_ZONA == @zona').groupby('QT_ELEITORES_INC_NM_SOCIAL')['QT_ELEITORES_PERFIL'].count()
    nome_social[zona] = result[1]
for zona in sorted(nome_social, key=lambda x:nome_social[x], reverse = True):
    print (f'Zona eleitoral {zona}: {nome_social[zona]}')

# Segunda parte: análise dos boletins de urna do 1o turno
<br>
<br>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
estado = pd.read_csv("bweb_1t_AM_051020221321.csv", 
                 usecols = '''NR_ZONA NR_SECAO NM_MUNICIPIO QT_COMPARECIMENTO QT_APTOS QT_COMPARECIMENTO 
                 QT_ABSTENCOES NR_VOTAVEL NM_VOTAVEL QT_VOTOS DS_CARGO_PERGUNTA'''.split(),
                 sep=";", on_bad_lines = 'skip', encoding='Latin 1')
municipio = estado.query('NM_MUNICIPIO == "MANAUS" and DS_CARGO_PERGUNTA == "Presidente"')
municipio = municipio.drop(columns = 'NM_MUNICIPIO DS_CARGO_PERGUNTA'.split())

In [ ]:
municipio.sample(20)

In [ ]:
municipio.groupby('NM_VOTAVEL')['QT_VOTOS'].sum().sort_values(ascending = False)

In [ ]:
votacao = {}
for zona in municipio.NR_ZONA.unique():
    result = municipio.query('NR_ZONA == @zona').groupby('NM_VOTAVEL')['QT_VOTOS'].sum()
    total = sum(result)
    votacao[zona] = {'Jair Bolsonaro': round(result["JAIR BOLSONARO"] / total * 100, 1),
                     'Lula': round(result["LULA"] / total * 100, 1), 
                     'Simone Tebet': round(result['SIMONE TEBET'] / total * 100, 1),
                     'Nulos':round(result['Nulo'] / total * 100, 1), 
                     'Brancos':round(result['Branco'] / total * 100, 1)
                    }
votacao = pd.DataFrame(votacao).transpose()
votacao

In [ ]:
votacao['Jair Bolsonaro'].sort_values(ascending = False)

In [ ]:
votacao['Lula'].sort_values(ascending = False)

In [ ]:
print (f"Porcentagem de comparecimento {municipio['QT_COMPARECIMENTO'].sum()/municipio['QT_APTOS'].sum()*100:.2f}%")